In [1]:
from tensorflow import keras
from tensorflow.keras.datasets import reuters
import numpy as np


from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 


max_words = 1000
maxlen = 500
emb_dim = 32

In [2]:
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words = max_words)

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(
    x_train,
    y_train, 
    test_size = 0.2,
    stratify = y_train,
    random_state = 34
)

# 패딩 
train_seq = pad_sequences(x_train, maxlen = maxlen)
val_seq = pad_sequences(x_val, maxlen = maxlen)


model = keras.Sequential()

model.add(keras.Input(shape = (maxlen,)))
model.add(keras.layers.Embedding(max_words, emb_dim))

model.add(keras.layers.Conv1D(128, 7, activation='relu', padding= "valid", strides=2, input_shape=[None, 1]))
model.add(keras.layers.GRU(20, dropout=0.3))

model.add(keras.layers.Dense(46, activation='softmax'))

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [3]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 500, 32)             │          32,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 247, 128)            │          28,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 20)                  │           9,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 46)                  │             966 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 70,766 (276.43 KB)

 Trainable params: 70,766 (276.43 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    optimizer= "adam", 
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)

es_cb = keras.callbacks.EarlyStopping(patience=8, restore_best_weights=True, monitor='val_loss')

history = model.fit(
    train_seq, 
    y_train, 
    epochs = 300,
    batch_size = 32, 
    validation_data = (val_seq, y_val), 
    callbacks = [es_cb]
)

Epoch 1/300
225/225 ━━━━━━━━━━━━━━━━━━━━ 33s 117ms/step - accuracy: 0.3818 - loss: 2.5435 - val_accuracy: 0.4886 - val_loss: 2.0685
Epoch 2/300
225/225 ━━━━━━━━━━━━━━━━━━━━ 23s 99ms/step - accuracy: 0.5346 - loss: 1.8539 - val_accuracy: 0.5854 - val_loss: 1.6724
Epoch 3/300
165/225 ━━━━━━━━━━━━━━━━━━━━ 5s 92ms/step - accuracy: 0.5806 - loss: 1.6341  

In [ ]:
plt.figure()

plt.plot(history.history["loss"], label = "train_loss")
plt.plot(history.history["val_loss"], label = "val_loss")

plt.plot(history.history["accuracy"], label = "train_accuracy")
plt.plot(history.history["val_accuracy"], label = "val_accuracy")

plt.legend()
plt.xlabel("epoch")
plt.show()

In [ ]:
test_seq = pad_sequences( x_test, maxlen = maxlen )
print(model.evaluate(test_seq, y_test))